# Examples for functions from utils.alg_syst_solver module.

In [1]:
import numpy as np
import time

import tt

from qttpdesolver.utils.general import MODE_NP, MODE_TT, MODE_SP, rel_err
from qttpdesolver.utils.spec_matr import findif
from qttpdesolver.utils.alg_syst_solver import Capture, alg_syst_solve

## Vectors and matrices for examples

The vectors and matrices are constructed here in numpy format and then are transformed to [quantized] tensor train format (the last ones are named according to mask *_tt).

In [2]:
d = 8
n = 2**d
h = 1./n
tau = 1.E-8
eps = 1.E-8

u = np.arange(n)+1.
u_tt = tt.tensor(u.reshape([2]*d, order='F'), tau)

A    = findif(d, n, h, tau, mode=MODE_NP)
A_tt = findif(d, n, h, tau, mode=MODE_TT)
A_sp = findif(d, n, h, tau, mode=MODE_SP)

rhs = A.dot(u)
rhs_tt = tt.tensor(rhs.reshape([2]*d, order='F'), tau)

## Examples

In [3]:
algss_par = {}

_t = time.time()
u_calc = alg_syst_solve(A, rhs, u0=None, eps=None, algss_par=algss_par, verb=False)
print 'Time: %-6.3f    '%(time.time()-_t),
print 'Rel. err: %-8.2e'%rel_err(u_calc, u)
print 'Parameters and additional results:'
for par_name in algss_par:
    print '%-20s : '%par_name, algss_par[par_name]

Time: 0.107      Rel. err: 2.83e-14
Parameters and additional results:
iters                :  None
solver               :  lstsq
max_dx               :  None
max_res              :  None
max_rank             :  None


In [4]:
algss_par = {}

_t = time.time()
u_calc = alg_syst_solve(A_sp, rhs, u0=None, eps=None, algss_par=algss_par, verb=False)
print 'Time: %-6.3f    '%(time.time()-_t),
print 'Rel. err: %-8.2e'%rel_err(u_calc, u)
print 'Parameters and additional results:'
for par_name in algss_par:
    print '%-20s : '%par_name, algss_par[par_name]

Time: 0.001      Rel. err: 0.00e+00
Parameters and additional results:
iters                :  None
solver               :  spsolve
max_dx               :  None
max_res              :  None
max_rank             :  None


In [5]:
algss_par = {}
algss_par['nswp'] = 20
algss_par['kickrank'] = 4
algss_par['local_prec'] = 'n'
algss_par['local_iters'] = 2
algss_par['local_restart'] = 40
algss_par['max_full_size'] = 50
algss_par['tau'] = eps

_t = time.time()
u_calc_tt = alg_syst_solve(A_tt, rhs_tt, None, eps, algss_par)
print 'Time: %-6.3f    '%(time.time()-_t),
print 'Rel. err: %-8.2e'%rel_err(u_calc_tt, u_tt)
print 'Parameters and additional results:'
for par_name in algss_par:
    print '%-20s : '%par_name, algss_par[par_name]

Time: 0.050      Rel. err: 1.44e-11
Parameters and additional results:
nswp                 :  20
local_restart        :  40
local_iters          :  2
max_rank             :  6
tau                  :  1e-08
iters                :  6
solver               :  amen
max_dx               :  0.0
kickrank             :  4
max_res              :  1.223e-09
max_full_size        :  50
local_prec           :  n


In [6]:
import os
c = Capture()
c.start()
os.system('echo 10')
print('20')
c.stop()
print c.get_out()
print c.get_out_len()
# >>> ['10\n', '20\n']
# >>> 2

['10\n', '20\n']
2
